In [1]:
import tensorflow as tf
import numpy as np
import pickle,sys
import datetime
tf.reset_default_graph()

<img src="files/narre_cnn.png" width="600">

In [2]:
def cnn_block(embedded_reviews,embedding_size,filter_sizes,num_filters,name='user_conv-maxpool'):
    pooled_outputs = []
    for i, filter_size in enumerate(filter_sizes):
        with tf.name_scope(name+"-%s" % filter_size):
                # Convolution Layer
            filter_shape = [filter_size, embedding_size, 1, num_filters]
            W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
            b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
            embedded_reviews = tf.reshape(embedded_reviews, [-1, review_len_u, embedding_size, 1])

            conv = tf.nn.conv2d(
                    embedded_reviews,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
            h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
            pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, review_len_u - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
            pooled_outputs.append(pooled)
    return pooled_outputs

<img src="files/narre_user.png" width="400">

In [3]:
def user_attetion_block(h_drop_u,iid_a,attention_size,embedding_id,num_filters_total,l2_loss):
    Wau = tf.Variable(
                tf.random_uniform([num_filters_total, attention_size], -0.1, 0.1), name='Wau')
    
    Wru = tf.Variable(
                tf.random_uniform([embedding_id, attention_size], -0.1, 0.1), name='Wru')
    
    Wpu = tf.Variable(
                tf.random_uniform([attention_size, 1], -0.1, 0.1), name='Wpu')
    
    bau = tf.Variable(tf.constant(0.1, shape=[attention_size]), name="bau")
    bbu = tf.Variable(tf.constant(0.1, shape=[1]), name="bbu")
#     self.iid_a = tf.nn.relu(tf.nn.embedding_lookup(iidW, self.input_reuid))
    u_j = tf.einsum('ajk,kl->ajl', tf.nn.relu(
                tf.einsum('ajk,kl->ajl', h_drop_u, Wau) + tf.einsum('ajk,kl->ajl', iid_a, Wru) + bau),
                                             Wpu)+bbu  # None*u_len*1

    u_a = tf.nn.softmax(u_j,1)  # none*u_len*1

    
    l2_loss += tf.nn.l2_loss(Wau)
    l2_loss += tf.nn.l2_loss(Wru)
    
    return u_a,l2_loss

def user_path(input_u, input_uid, input_reuid, iidW, dropout_keep_prob,review_num_u,user_num,user_vocab_size,
              attention_size,embedding_size,embedding_id,n_latent,filter_sizes,num_filters,l2_loss):
    with tf.name_scope("user_embedding"):
        W1 = tf.Variable(
                    tf.random_uniform([user_vocab_size, embedding_size], -1.0, 1.0),
                    name="W1")
        embedded_user = tf.nn.embedding_lookup(W1, input_u)
        embedded_users = tf.expand_dims(embedded_user, -1)
        
    pooled_outputs_u = cnn_block(embedded_users,embedding_size,filter_sizes,num_filters,name='user_conv-maxpool')       
    num_filters_total = num_filters * len(filter_sizes)
    h_pool_u = tf.concat(pooled_outputs_u,axis=-1)
    h_pool_flat_u = tf.reshape(h_pool_u, [-1, review_num_u, num_filters_total])
        
    with tf.name_scope("dropout"):
        h_drop_u = tf.nn.dropout(h_pool_flat_u, 1.0)
            
    with tf.name_scope("attention") as attention_scope:
        iid_a = tf.nn.relu(tf.nn.embedding_lookup(iidW, input_reuid))
        u_a,l2_loss = user_attetion_block(h_drop_u,iid_a,attention_size,embedding_id,num_filters_total,l2_loss)
            
    with tf.name_scope("add_reviews"):
        u_feas = tf.reduce_sum(tf.multiply(u_a, h_drop_u), 1)
        u_feas = tf.nn.dropout(u_feas, dropout_keep_prob)
        
    with tf.name_scope("get_fea") as get_fea_scope:
        uidmf = tf.Variable(tf.random_uniform([user_num + 2, embedding_id], -0.1, 0.1), name="uidmf")
        uid = tf.nn.embedding_lookup(uidmf,input_uid)    
        uid = tf.reshape(uid,[-1,embedding_id])
            
        Wu = tf.Variable(
                tf.random_uniform([num_filters_total, n_latent], -0.1, 0.1), name='Wu')
        bu = tf.Variable(tf.constant(0.1, shape=[n_latent]), name="bu")
        
        u_feas = tf.matmul(u_feas, Wu)+uid + bu

    return u_a,u_feas,l2_loss,attention_scope,get_fea_scope

<img src="files/narre_item.png" width="400">

In [4]:
def item_attetion_block(h_drop_i,uid_a,attention_size,embedding_id,num_filters_total,l2_loss):
    Wai = tf.Variable(
                tf.random_uniform([num_filters_total, attention_size], -0.1, 0.1), name='Wai')
    
    Wri = tf.Variable(
                tf.random_uniform([embedding_id, attention_size], -0.1, 0.1), name='Wri')
    
    Wpi = tf.Variable(
                tf.random_uniform([attention_size, 1], -0.1, 0.1), name='Wpi')
    
    bai = tf.Variable(tf.constant(0.1, shape=[attention_size]), name="bai")
    bbi = tf.Variable(tf.constant(0.1, shape=[1]), name="bbi")
    
#     self.uid_a = tf.nn.relu(tf.nn.embedding_lookup(uidW, self.input_reiid))
    i_j =tf.einsum('ajk,kl->ajl', tf.nn.relu(
                tf.einsum('ajk,kl->ajl', h_drop_i, Wai) + tf.einsum('ajk,kl->ajl', uid_a, Wri) + bai),
                                             Wpi)+bbi

    i_a = tf.nn.softmax(i_j,1)  # none*len*1
    
    l2_loss += tf.nn.l2_loss(Wri)
    l2_loss += tf.nn.l2_loss(Wai)
    
    return i_a,l2_loss

def item_path(input_i, input_iid, input_reiid, uidW, dropout_keep_prob,review_num_i,item_num,item_vocab_size,
              attention_size,embedding_size,embedding_id,n_latent,filter_sizes,num_filters,l2_loss,attention_scope,get_fea_scope):
    with tf.name_scope("item_embedding"):
        W2 = tf.Variable(
                tf.random_uniform([item_vocab_size, embedding_size], -1.0, 1.0),
                name="W2")
        embedded_item = tf.nn.embedding_lookup(W2, input_i)
        embedded_items = tf.expand_dims(embedded_item, -1)
        
    pooled_outputs_i = cnn_block(embedded_items,embedding_size,filter_sizes,num_filters,name='item_conv-maxpool')      
    num_filters_total = num_filters * len(filter_sizes)
    h_pool_i = tf.concat(pooled_outputs_i,axis=-1)
    h_pool_flat_i = tf.reshape(h_pool_i, [-1, review_num_i, num_filters_total])
        
    with tf.name_scope("dropout"):
        h_drop_i = tf.nn.dropout(h_pool_flat_i, 1.0)
            
    with tf.name_scope(attention_scope):
        uid_a = tf.nn.relu(tf.nn.embedding_lookup(uidW, input_reiid))
        i_a,l2_loss = item_attetion_block(h_drop_i,uid_a,attention_size,embedding_id,num_filters_total,l2_loss)
            
    with tf.name_scope("add_reviews"):
        i_feas = tf.reduce_sum(tf.multiply(i_a, h_drop_i), 1)
        i_feas = tf.nn.dropout(i_feas, dropout_keep_prob)
           
        
    with tf.name_scope(get_fea_scope):
        iidmf = tf.Variable(tf.random_uniform([item_num + 2, embedding_id], -0.1, 0.1), name="iidmf")
        iid = tf.nn.embedding_lookup(iidmf,input_iid)
        iid = tf.reshape(iid,[-1,embedding_id])
            
        Wi = tf.Variable(
                tf.random_uniform([num_filters_total, n_latent], -0.1, 0.1), name='Wi')
        bi = tf.Variable(tf.constant(0.1, shape=[n_latent]), name="bi")
        i_feas = tf.matmul(i_feas, Wi) +iid+ bi

    return i_a,i_feas,l2_loss

<img src="files/narre.png" width=600>

In [5]:
class NARRE(object):
    def __init__(
            self, review_num_u, review_num_i, review_len_u, review_len_i, user_num, item_num, num_classes,
            user_vocab_size, item_vocab_size, n_latent, embedding_id, attention_size,
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        self.input_u = tf.placeholder(tf.int32, [None, review_num_u, review_len_u], name="input_u")
        self.input_i = tf.placeholder(tf.int32, [None, review_num_i, review_len_i], name="input_i")
        self.input_reuid = tf.placeholder(tf.int32, [None, review_num_u], name='input_reuid')
        self.input_reiid = tf.placeholder(tf.int32, [None, review_num_i], name='input_reuid')
        self.input_y = tf.placeholder(tf.float32, [None, 1], name="input_y")
        self.input_uid = tf.placeholder(tf.int32, [None, 1], name="input_uid")
        self.input_iid = tf.placeholder(tf.int32, [None, 1], name="input_iid")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
        self.drop0 = tf.placeholder(tf.float32, name="dropout0")
        iidW = tf.Variable(tf.random_uniform([item_num + 2, embedding_id], -0.1, 0.1), name="iidW")
        uidW = tf.Variable(tf.random_uniform([user_num + 2, embedding_id], -0.1, 0.1), name="uidW")

        l2_loss = tf.constant(0.0)
        
        
        self.u_a,self.u_feas, l2_loss,attention_scope,get_fea_scope = user_path(self.input_u, self.input_uid, self.input_reuid, iidW, self.dropout_keep_prob,review_num_u,user_num,user_vocab_size,
              attention_size,embedding_size,embedding_id,n_latent,filter_sizes,num_filters,l2_loss)

        self.i_a,self.i_feas, l2_loss = item_path(self.input_i, self.input_iid, self.input_reiid, uidW, self.dropout_keep_prob,review_num_i,item_num,item_vocab_size,
              attention_size,embedding_size,embedding_id,n_latent,filter_sizes,num_filters,l2_loss,attention_scope,get_fea_scope)


        with tf.name_scope('ncf'):

            self.FM = tf.multiply(self.u_feas, self.i_feas)
            self.FM = tf.nn.relu(self.FM)

            self.FM=tf.nn.dropout(self.FM,self.dropout_keep_prob)

            Wmul=tf.Variable(
                tf.random_uniform([n_latent, 1], -0.1, 0.1), name='wmul')

            self.mul=tf.matmul(self.FM,Wmul)
            self.score=tf.reduce_sum(self.mul,1,keep_dims=True)

            self.uidW2 = tf.Variable(tf.constant(0.1, shape=[user_num + 2]), name="uidW2")
            self.iidW2 = tf.Variable(tf.constant(0.1, shape=[item_num + 2]), name="iidW2")
            self.u_bias = tf.gather(self.uidW2, self.input_uid)
            self.i_bias = tf.gather(self.iidW2, self.input_iid)
            self.Feature_bias = self.u_bias + self.i_bias

            self.bised = tf.Variable(tf.constant(0.1), name='bias')

            self.predictions = self.score + self.Feature_bias + self.bised

        with tf.name_scope("loss"):
            losses = tf.nn.l2_loss(tf.subtract(self.predictions, self.input_y))

            self.loss = losses + l2_reg_lambda * l2_loss

        with tf.name_scope("accuracy"):
            self.mae = tf.reduce_mean(tf.abs(tf.subtract(self.predictions, self.input_y)))
            self.accuracy =tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.predictions, self.input_y))))


In [6]:
np.random.seed(2017)
random_seed = 2017

word2vec = 'NARRE-master/data/google.bin'
valid_data = "NARRE-master/data/music/music.test"
para_data = "NARRE-master/data/music/music.para"
train_data = "NARRE-master/data/music/music.train"

embedding_dim = 300
filter_sizes = "3"
num_filters = 100
dropout_keep_prob = 0.5
l2_reg_lambda = 0.001

batch_size =  256
num_epochs = 40

allow_soft_placement = True
log_device_placement = False

In [7]:
%%time
%cd /var/nvidia/vishal/
print("Loading data...")
pkl_file = open(para_data, 'rb')

para = pickle.load(pkl_file)
user_num = para['user_num']
item_num = para['item_num']
review_num_u = para['review_num_u']
review_num_i = para['review_num_i']
review_len_u = para['review_len_u']
review_len_i = para['review_len_i']
vocabulary_user = para['user_vocab']
vocabulary_item = para['item_vocab']
train_length = para['train_length']
test_length = para['test_length']
u_text = para['u_text']
i_text = para['i_text']

pkl_file = open(valid_data, 'rb')
test_data = pickle.load(pkl_file)
test_data = np.array(test_data)
data_size_test = len(test_data)
pkl_file.close()

/var/nvidia/vishal
Loading data...
CPU times: user 12.8 s, sys: 1.4 s, total: 14.2 s
Wall time: 14.2 s


In [8]:
pad_id = 1
inv_vocabulary_user = {v:k for k,v in vocabulary_user.iteritems()}
inv_vocabulary_item = {v:k for k,v in vocabulary_item.iteritems()}

def get_text_from_review_ids(arr,vocab=inv_vocabulary_user):
    review = ''
    for a in arr:
        if(a==pad_id):
            return review.rstrip(" ")
        else:
            review+=vocab[a]+" "
    return review.rstrip(" ")

In [9]:
def dev_step_with_attention(u_batch, i_batch, uid, iid, reuid, reiid, y_batch, writer=None):
    """
    Evaluates model on a dev set

    """
    feed_dict = {
        deep.input_u: u_batch,
        deep.input_i: i_batch,
        deep.input_y: y_batch,
        deep.input_uid: uid,
        deep.input_iid: iid,
        deep.input_reuid: reuid,
        deep.input_reiid: reiid,
        deep.drop0: 1.0,
        deep.dropout_keep_prob: 1.0
    }
    loss, accuracy, mae, u_a, i_a = sess.run(
        [deep.loss, deep.accuracy, deep.mae,deep.u_a, deep.i_a,],
        feed_dict)
    return [loss, accuracy, mae, u_a, i_a]

In [10]:
tf.reset_default_graph()
with tf.Session() as sess:
    deep = NARRE(
                review_num_u=review_num_u,
                review_num_i=review_num_i,
                review_len_u=review_len_u,
                review_len_i=review_len_i,
                user_num=user_num,
                item_num=item_num,
                num_classes=1,
                user_vocab_size=len(vocabulary_user),
                item_vocab_size=len(vocabulary_item),
                embedding_size=embedding_dim,
                embedding_id=32,
                filter_sizes=list(map(int, filter_sizes.split(","))),
                num_filters=num_filters,
                l2_reg_lambda=l2_reg_lambda,
                attention_size=32,
                n_latent=32)
    saver = tf.train.Saver()
    saver.restore(sess, "narre_model/")
    ll_test = int(len(test_data) / batch_size) + 1
    for batch_num in range(10,11):
        start_index = batch_num * batch_size
        end_index = min((batch_num + 1) * batch_size, data_size_test)
        data_test = test_data[start_index:end_index]

        userid_valid, itemid_valid, reuid, reiid, y_valid = zip(*data_test)
        
#         print(userid_valid[0][0],itemid_valid)
        u_valid = []
        i_valid = []
        for i in range(len(userid_valid)):
            u_valid.append(u_text[userid_valid[i][0]])
            i_valid.append(i_text[itemid_valid[i][0]])
        u_valid = np.array(u_valid)
        i_valid = np.array(i_valid)

        loss, accuracy, mae,u_a,i_a = dev_step_with_attention(u_valid, i_valid, userid_valid, itemid_valid, reuid, reiid,
                                                           y_valid)
        
        print("instance: ",userid_valid[0][0],itemid_valid[0][0],y_valid[0][0])
        print("Reviews given by user: ",userid_valid[0][0],"/n")
        for i,arr in enumerate(u_text[userid_valid[0][0]]):
            review = get_text_from_review_ids(arr,inv_vocabulary_user)
            if(review):
                print(i,u_a[0][i][0],review)
        print("Reviews recieved on item: ",itemid_valid[0][0],"/n")
        for i,arr in enumerate(i_text[itemid_valid[0][0]]):
            review = get_text_from_review_ids(arr,inv_vocabulary_item)
            if(review):
                print(i,i_a[0][i],review)
                
                

W1021 20:53:31.858580 140155372729280 deprecation.py:506] From <ipython-input-3-ba0eb96b92fd>:41: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1021 20:53:32.109607 140155372729280 deprecation.py:506] From <ipython-input-5-ff579f09b996>:39: calling reduce_sum_v1 (from tensorflow.python.ops.math_ops) with keep_dims is deprecated and will be removed in a future version.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
W1021 20:53:32.290285 140155372729280 deprecation.py:323] From /dev/shm/vishal/vishal.Linux_debian_9_3.py27.keras2/lib/python2.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs 

('instance: ', 1562, 1392, 5.0)
('Reviews given by user: ', 1562, '/n')
(0, 0.07464726, u'from all beegees albums i would say this one makes you the most curious once you purchase it songs like never say never again suddenly first of may sound of love and melody fair are incredible but the rest of the songs make it sound like they just sat down in their studio and whatever popped into their head they produced right away but some of the songs are still pretty good those three classical tunes i really don t listen to but not a bad cd for songs')
(1, 0.36030912, u'my favorite songs on this cd happen to be the weirdest named ones birdie told me the earnest of being george and lemons never forget that nd one i don t have a slight idea wut it means but it has a cool beat to it birdie told me and lemons never forget are mid paced songs and they re both catchy the hit quot massachusetts quot is boring to me it just ain t my style but world is robin does great and that song maurice s song is wi

# 90% of this code is from https://github.com/chenchongthu/NARRE

In [ ]:
#  Show attention score for users and items with less length of review